## Food/Diet Recommender System

#### Group04 ##

**Tutorial section: TT5L**

| | Student ID     | Student Name                |
|--:|:-------------|:---------------------------:|
| 1 | 1201103207   | Melody Koh Si Jie           | 
| 2 | 1211103427   | Law Chin Keat               |
| 3 | 1211102976   | Lee Le Xuan                 | 
| 4 | 1211200755   | Wendy Tan Wei Wei           | 

### 1.0 Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns